# <center>ANALYSIS ON VARIANT PRICES OF BOOKS BASED ON THEIR CATEGORIES</center>

<h3><center> Part 1:   Data Cleaning  </center></h3>

<h3>Brief Description</h3>

The Dataset in used for this analysis was gotten from an hosted competition in www.machinehack.com, the objective of the competition is to build a machine learnig model to predict the price of books based on a given set of features. these books are of different genres, from thousand of authors

features of the data provided:
<ul>
    <li>Title</li>
    <li>Author</li>
    <li>Edition</li>
    <li>Reviews</li>
    <li>Ratings</li>
    <li>Synopsis</li>
    <li>Genre</li>
    <li>BookCategory</li>
    <li>Price</li>

</ul>


The above features as provided in the data, appears messy, hence conducting any form of data analysis or modelling with the data would be challenging, so there is need for proper data cleaning before deriving insights.

This Notebook focuses more on data cleaning.|


<h4>Importing neccessary packages and reading in the data</h4>

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', None)

In [2]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Test.xlsx')
sub = pd.read_excel('Sample_Submission.xlsx')

In [3]:
print(train.shape)
print(test.shape)

(6237, 9)
(1560, 8)


In [4]:
train.head(3)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00


After proper examination of each columns, our columns of intrest are
<ul>
    <li>Edition</li>
    <li>Reviews</li>
    <li>Ratings</li>
    <li>Book Category</li>
    <li>Price</li>
</ul>

So our next couple of cells focuses on how the columns of intrest were cleaned and prepared for analysis

### Cleaning Edition Column

In [6]:
#deriving date of edition columns for each books written by provided authors
import re
date_regex = re.compile(r'([\d]+[\D]+[\d]{4})', flags= re.IGNORECASE)

train['date_of_edition'] = train.loc[:, 'Edition'].str.extract(date_regex, expand = False)
test['date_of_edition'] = test.loc[:, 'Edition'].str.extract(date_regex, expand = False)

In [7]:
#deriving column for the each book cover type
import re
book_regex = re.compile(r'(Paperback|Hardcover)', flags= re.IGNORECASE)

train['book_cover_type'] = train.loc[:, 'Edition'].str.extract(book_regex, expand = False)
test['book_cover_type'] = test.loc[:, 'Edition'].str.extract(book_regex, expand = False)

In [8]:
train.head(2)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,date_of_edition,book_cover_type
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,10 Mar 2016,Paperback
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,7 Nov 2012,Paperback


### Cleaning up Reviews column

In [10]:
train.loc[0,'Reviews']

'4.0 out of 5 stars'

In [11]:
#Extracting columns for reviews
train['review_rate'] = train.loc[:, 'Reviews'].str.extract(pat = '(\d.\d)', expand = False)
test['review_rate'] = test.loc[:, 'Reviews'].str.extract(pat = '(\d.\d)', expand = False)

### Cleaning up Rating Column

In [12]:
train.loc[0,'Ratings']

'8 customer reviews'

In [13]:
#Extracting columns for ratingss
train['readers_rating'] = train.loc[:, 'Ratings'].str.extract(pat = '(\d)', expand  = False)
test['readers_rating'] = test.loc[:, 'Ratings'].str.extract(pat = '(\d)', expand  = False)

### Cleaning up Date of Edition Column

In [14]:
import datetime as dt
train['date_of_edition'] = train['date_of_edition'].astype('datetime64[ns]')
test['date_of_edition'] = test['date_of_edition'].astype('datetime64[ns]')

In [15]:
#deriving the year
train['year'] = train.loc[:, 'date_of_edition'].dt.year
test['year'] = test.loc[:, 'date_of_edition'].dt.year

#deriving the month
train['month'] = train.loc[:, 'date_of_edition'].dt.month
test['month'] = test.loc[:, 'date_of_edition'].dt.month

In [16]:
train.head(1)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,date_of_edition,book_cover_type,review_rate,readers_rating,year,month
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.0,2016-03-10,Paperback,4.0,8,2016.0,3.0


After proper cleaning we have been able to create neccessary columns to derive insight from the data provided. list of columns created include:
<ul>
    <li>date_of_edition</li>
    <li>book_cover_type</li>
    <li>review_rate</li>
    <li>readers_rating</li>
    <li>year</li>
    <li>month</li>
    
</ul>

Our next cells shows how we handled missing values in each cloumns after dropping irrelevant columns for the analysis

### Handling Missing Values 

In [19]:
train.isnull().sum()

Title                0
Author               0
Edition              0
Reviews              0
Ratings              0
Synopsis             0
Genre                0
BookCategory         0
Price                0
date_of_edition    452
book_cover_type     61
review_rate          0
readers_rating       0
year               452
month              452
dtype: int64

we have four columns with missing values [date_of_edition, book_cover_type, year and month]

<b>Note:</b> handling of missing values can be of different strategy, based on the analysis involved, on this project am replacing null values in each columns with "not specified"

In [20]:
train.columns

Index(['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price', 'date_of_edition', 'book_cover_type',
       'review_rate', 'readers_rating', 'year', 'month'],
      dtype='object')

In [21]:
coln = ['Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre']
for x in coln:
    train = train.drop([x], axis = 1)
    test = test.drop([x], axis = 1)

In [23]:
train.head(2)

,Title,Author,BookCategory,Price,date_of_edition,book_cover_type,review_rate,readers_rating,year,month
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,Action & Adventure,220.00,2016-03-10,Paperback,4.0,8,2016.0,3.0
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Biographies, Diaries & True Accounts",202.93,2012-11-07,Paperback,3.9,1,2012.0,11.0


In [24]:
fill_miss = {'date_of_edition': 'not specified', 'book_cover_type': 'not specified',
            'year': 'not specified', 'month': 'not specified'}

train.fillna(fill_miss, inplace = True)
test.fillna(fill_miss, inplace = True)

train.isnull().sum()

Title              0
Author             0
BookCategory       0
Price              0
date_of_edition    0
book_cover_type    0
review_rate        0
readers_rating     0
year               0
month              0
dtype: int64

In [30]:
#save our clean data set to csv file and excel file for further analysis on PowerBi software and also on python software
train.to_csv('new_train.csv', index = False)
test.to_csv('new_test.csv', index = False)

train.to_excel('trainexcel.xlsx', index = False)
test.to_excel('testexcel.xlsx', index = False)